In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

2024-06-19 22:59:04.600849: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Загрузка датасета, подготовка обучающего и тестового набора

mnist_bldr = tfds.builder('mnist', data_dir='./dataset')
mnist_bldr.download_and_prepare()
datasets = mnist_bldr.as_dataset(shuffle_files=False)
mnist_train_orig = datasets['train']
mnist_test_orig = datasets['test']

In [3]:
# Настройка типов данных в наборах и размера пакетов

BUFFER_SIZE = 10000
BATCH_SIZE = 64
NUM_EPOCHS = 20

mnist_train = mnist_train_orig.map(
    lambda item: (
        tf.cast(item["image"], tf.float32) / 255,
        tf.cast(item["label"], tf.int32),
    )
)
mnist_test = mnist_test_orig.map(
    lambda item: (
        tf.cast(item["image"], tf.float32) / 255,
        tf.cast(item["label"], tf.int32),
    )
)
tf.random.set_seed(1)

mnist_train = mnist_train.shuffle(
    buffer_size=BUFFER_SIZE, reshuffle_each_iteration=False
)

mnist_valid = mnist_train.take(10000).batch(BATCH_SIZE)
mnist_train = mnist_train.skip(10000).batch(BATCH_SIZE)

In [4]:
# Сборка сверточной нейронной сети

model = tf.keras.Sequential()
model.add(
    tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=(5, 5),
        strides=(1, 1),
        padding="same",
        data_format="channels_last",
        name="conv_1",
        activation="relu",
    )
)
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), name="pool_1"))
model.add(
    tf.keras.layers.Conv2D(
        filters=64,
        kernel_size=(5, 5),
        strides=(1, 1),
        padding="same",
        name="conv_2",
        activation="relu",
    )
)
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), name="pool_2"))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=1024, name="fc_1", activation="relu"))
model.add(tf.keras.layers.Dropout(rate=0.5))
model.add(tf.keras.layers.Dense(units=10, name="fc_2", activation="softmax"))

tf.random.set_seed(1)
model.build(input_shape=(None, 28, 28, 1))
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [ ]:
# Обучение нейронной сети

model.fit(mnist_train, epochs=NUM_EPOCHS, validation_data=mnist_valid, shuffle=True)

In [ ]:
# Сохранение обученной модели в файл

model.save('main_model.keras')